# Global mean Sea Surface Temperatures

In [21]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
from matplotlib.dates import DateFormatter
import platform
import os

from dask.distributed import Client
from dask import delayed
import dask

import cartopy.crs as ccrs
import cartopy
import cartopy.feature as cfeature

from shapely.geometry import Polygon, Point
from shapely.ops import transform
import pyproj

plt.rcParams['figure.figsize'] = (10, 4)

In [19]:
# Define a transformation to ensure the polygon's CRS matches
# Transform the polygon to match the DataArray CRS if needed
def transform_polygon(polygon, src_crs='epsg:4326', tgt_crs='epsg:4326'):
    proj = pyproj.Transformer.from_proj(pyproj.Proj(src_crs), pyproj.Proj(tgt_crs), always_xy=True)
    return transform(lambda x, y: proj.transform(x, y), polygon)

In [18]:
def point_in_polygon(lon, lat, polygon):
    point = Point(lon, lat)
    return polygon.contains(point)

# Settings

In [2]:
year1= 1982
year2= 2025

In [3]:
# Settings compute de climatoloy
yearC1='1982'
yearC2='1992'

# Inicio

In [4]:
HOME=os.environ['HOME']   
f = open(HOME+'/.env', 'r')
for line in f.readlines():
    Name=line.strip().split('=')[0]
    Content=line.strip().split('=')[-1]
    if Name=='dirData' or Name=='dirAnalisis':
        exec(Name + "=" + "'" + Content + "'")
f.close()

In [5]:
if os.uname().nodename.lower().find('eemmmbp') != -1:
    base_file = dirData + '/Satelite/noaa.oisst.v2.highres/NC/sst.day.mean'
    dataDir   = dirAnalisis + '/SSTGlobalAnalysis/data'
elif os.uname().nodename.lower().find('sagams') != -1:
    base_file = dirData + '/Satelite/noaa.oisst.v2.highres/NC/sst.day.mean'
    dataDir   = dirAnalisis + '/SSTGlobalAnalysis/data'
elif os.uname().nodename.lower().find('rossby') != -1:
    base_file = dirData + '/Satelite/noaa.oisst.v2.highres/NC/sst.day.mean'
    dataDir   = dirAnalisis + '/SSTGlobalAnalysis/data'

In [6]:
base_file

'/data/pvb/Satelite/noaa.oisst.v2.highres/NC/sst.day.mean'

In [7]:
Titulos = ['Oceano Global','Hemisferio norte','Hemisferio sur',
           'Atlántico Norte','Iberian Canary Basin',
           'Demarcación marina levantino-balear', 'Demarcación marina noratlántica','Demarcación marina canaria','Demarcación sudatlántica','Demarcación Estrecho y Alborán']
Titulos_short = ['GO','NH','SH',
                 'NAtl','IBICan',
                 'LEB', 'NOR','CAN','SUD','ESA']

## Load data

In [8]:
it=7

In [9]:
titulo = Titulos[it]
titulo_short = Titulos_short[it]

In [10]:
print(titulo)

Demarcación marina canaria


In [11]:
files = [f'{base_file}.{year}.nc' for year in range(year1, year2+1)]

In [ ]:
DS = xr.open_mfdataset(files)

In [58]:
# Load the data from the .txt file
demCoord = []
longDem, latiDem = [], []
with open('./data/Demarcacion'+titulo_short+'.txt', 'r') as f:
    for line in f:
        # Split the line by whitespace and append the values
        longitude, latitude = map(float, line.split())
        longitude=longitude+360
        longDem.append(longitude)
        latiDem.append(latitude)
        demCoord.append((longitude,latitude))
demPolygon = Polygon(demCoord)    
demPolygon_transformed = transform_polygon(demPolygon)

In [59]:
demCoord

[(348.0276191000022, 30.6505730980622),
 (348.1873529000022, 30.3433248980736),
 (348.1865538000023, 30.3047879980752),
 (347.9229216000023, 29.430863998109),
 (347.7823706000023, 29.1274278981211),
 (347.6661783000023, 28.9976227981264),
 (347.5543025526444, 28.9013836902643),
 (347.4886522373837, 28.8557149434319),
 (347.3205116306273, 28.7346670645906),
 (347.3058826258124, 28.7238912706966),
 (347.1618342000025, 28.6331306981414),
 (346.8958669000024, 28.5175740981462),
 (346.7608119000025, 28.4190522981505),
 (346.6109801000026, 28.1890237981602),
 (346.5558229000025, 28.0776318981649),
 (346.3608754000026, 27.8629899981741),
 (346.1558379000026, 27.6624464981829),
 (346.0338916000026, 27.5244839981888),
 (345.8687462000026, 27.3875628981948),
 (345.6522340000027, 27.2331415982018),
 (345.4434232000027, 27.2118465982027),
 (345.0813157000028, 27.029986098211),
 (344.8897331951065, 26.9167403768836),
 (344.40681094549143, 26.6315354626052),
 (343.9520623000031, 26.3762834982409),
 

In [ ]:
if it == 4 :
    sst = DS.sst.sel(lat=slice(20, 50),lon=slice(325,360))
    basins = xr.open_dataset(dataDir+'/basins.nc')
    basin_surf = basins.basin[0]
    basin_surf_interp = basin_surf.interp_like(sst, method='nearest')
    sst = sst.where((basin_surf_interp==1) ,drop=Truex)


if titulo_short == 'LEB':
    slicelatitude=slice(35.5,42.75)
    slicelongitude=slice(358,368)
        sst=DS.sst.sel(lat=slicelatitude).sel(lon=slicelongitude)
    mask = np.array([[point_in_polygon(lon,lat,demPolygon_transformed) 
         for lon in sst.lon.values] 
         for lat in sst.lat.values])
    sst_unmasked = sst
    sst = sst.where(mask)
    print('>>>>> '+titulo)        

elif  titulo_short == 'NOR':
    slicelatitude=slice(41.5,46.9)
    slicelongitude=slice(346,360)
    sst=DS.sst.sel(lat=slicelatitude).sel(lon=slicelongitude)
    mask = np.array([[point_in_polygon(lon,lat,demPolygon_transformed) 
         for lon in sst.lon.values] 
         for lat in sst.lat.values])
    sst_unmasked = sst
    sst = sst.where(mask)
    print('>>>>> '+titulo)        
        
elif  titulo_short == 'CAN':
    slicelatitude=slice(24,32.5)
    slicelongitude=slice(335,350)
    sst=DS.sst.sel(lat=slicelatitude).sel(lon=slicelongitude)
    mask = np.array([[point_in_polygon(lon,lat,demPolygon_transformed) 
         for lon in sst.lon.values] 
         for lat in sst.lat.values])
    sst_unmasked = sst
    sst = sst.where(mask)
    print('>>>>> '+titulo)    

elif  titulo_short == 'SUD':
    slicelatitude=slice(35.5,37.5)
    slicelongitude=slice(352,354.5)
    sst=DS.sst.sel(lat=slicelatitude).sel(lon=slicelongitude)
    mask = np.array([[point_in_polygon(lon,lat,demPolygon_transformed) 
         for lon in sst.lon.values] 
         for lat in sst.lat.values])
    sst_unmasked = sst
    sst = sst.where(mask)
    print('>>>>> '+titulo)

elif  titulo_short == 'ESA':
    slicelatitude=slice(35.5,37)
    slicelongitude=slice(354,358.5)
    sst=DS.sst.sel(lat=slicelatitude).sel(lon=slicelongitude)
    mask = np.array([[point_in_polygon(lon,lat,demPolygon_transformed) 
         for lon in sst.lon.values] 
         for lat in sst.lat.values])
    sst_unmasked = sst
    sst = sst.where(mask)
    print('>>>>> '+titulo)
        




>>>>> Demarcación marina canaria


In [63]:
print('>>>>> '+titulo)

>>>>> Demarcación marina canaria


## Daily analisis

In [64]:
print('>>>>> Daily'+titulo+titulo_short)

>>>>> DailyDemarcación marina canariaCAN


In [65]:
## Calculate mean weigthtened
weights = np.cos(np.deg2rad(sst.lat))
weights = weights/weights.max()
weights.name = "weights"
sst_weighted = sst.weighted(weights)
sst_wmean = sst_weighted.mean(("lon", "lat"),skipna=True).load()

In [66]:
## Create monthly climatology
sst_clim = sst.sel(time=slice(yearC1,yearC2)).groupby('time.dayofyear').mean(dim='time').load();

In [67]:
## Create anomaly
sst_anom = sst.groupby('time.dayofyear') - sst_clim

/opt/conda/envs/ocean/lib/python3.11/site-packages/xarray/core/indexing.py:1593: PerformanceWarning: Slicing with an out-of-order index is generating 44 times more chunks
  return self.array[key]


In [68]:
## Calculate global mean anomaly
weights = np.cos(np.deg2rad(sst.lat))
weights = weights/weights.max()
weights.name = "weights"
sst_anom_weighted = sst_anom.weighted(weights)
sst_anom_wmean = sst_anom_weighted.mean(("lon", "lat"),skipna=True).load()

In [69]:
## Save in netcdf
sst_wmean.to_netcdf(dataDir+'/sstd_mean_'+titulo_short+'.nc',mode='w')   
sst_anom_wmean.to_netcdf(dataDir+'/sstd_anom_mean_'+titulo_short+'.nc',mode='w')

In [70]:
if titulo_short=='GO' or titulo_short=='NAtl' or titulo_short=='LEB' or titulo_short=='CAN' or titulo_short=='NOR' or titulo_short=='SUD' or titulo_short=='ESA' or titulo_short=='IBICan':
        sst_anom_LD=sst_anom[-1,:,:]
        sst_anom_LD.to_netcdf(dataDir+'/sstLD_anom_'+titulo_short+'.nc',mode='w')

# Monthly analisis

In [ ]:
sst = sst.resample(time='ME').mean(dim='time',skipna=True).load()

In [ ]:
## Calculate global mean weigthtened
weights = np.cos(np.deg2rad(sst.lat))
weights = weights/weights.max()
weights.name = "weights"
sst_weighted = sst.weighted(weights)
sst_wmean = sst_weighted.mean(("lon", "lat"),skipna=True).load()
    

In [ ]:
## Create monthly climatology
sst_clim = sst.sel(time=slice(yearC1,yearC2)).groupby('time.month').mean(dim='time').load();

In [ ]:
## Create anomaly
print('    > Compute anomaly mean')
sst_anom = sst.groupby('time.month') - sst_clim
sst_anom.load();

In [ ]:
##Calculate global mean weigthtened
print('    > Compute weigthtened mean')
weights = np.cos(np.deg2rad(sst.lat))
weights = weights/weights.max()
weights.name = "weights"
sst_anom_weighted = sst_anom.weighted(weights)
sst_anom_wmean = sst_anom_weighted.mean(("lon", "lat"),skipna=True).load()
sst_anom_wmean_rolling = sst_anom_wmean.rolling(time=12,center=True).mean()

In [ ]:
##Save in netcdf
print('    > to netcdf')
sst_anom.to_netcdf(dataDir+'/sstm_anom_'+titulo_short+'.nc',mode='w')
sst_wmean.to_netcdf(dataDir+'/sstm_mean_'+titulo_short+'.nc',mode='w')
sst_anom_wmean.to_netcdf(dataDir+'/sstm_anom_mean_'+titulo_short+'.nc',mode='w')